In [ ]:
!pip install datasets
!pip install -U transformers
!pip install transformers peft torch
!pip install --upgrade peft

In [2]:
import os
import shutil
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import warnings
import pandas as pd
from torchvision import transforms
import glob
from tqdm import tqdm
from urllib.request import urlopen
from PIL import Image

from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
import pandas as pd
from datasets import Dataset

import time

# **Random seed 고정**

In [3]:
import random

seed = 40
deterministic = True

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
if deterministic:
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False
warnings.filterwarnings('ignore')

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# **google drive mount**

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **데이터 다운로드**

In [6]:
from datasets import load_dataset

data = load_dataset("tatsu-lab/alpaca")

# train-test split
dataset = data["train"].train_test_split(test_size=0.3, seed=42)

README.md:   0%|          | 0.00/7.47k [00:00<?, ?B/s]

(…)-00000-of-00001-a09b74b3ef9c3b56.parquet:   0%|          | 0.00/24.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/52002 [00:00<?, ? examples/s]

In [7]:
#data 형식 확인
dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'text'],
        num_rows: 36401
    })
    test: Dataset({
        features: ['instruction', 'input', 'output', 'text'],
        num_rows: 15601
    })
})

drive에 데이터 저장

In [ ]:
# Drive에 저장할 경로
train_save_path = '/content/drive/My Drive/alpaca_train_dataset.csv'
test_save_path = '/content/drive/My Drive/alpaca_test_dataset.csv'

# CSV 파일로 저장
dataset["train"].to_csv(train_save_path)
dataset["test"].to_csv(test_save_path)
print(f"train_dataset saved to {train_save_path} , test_dataset saved to {test_save_path}")

drive에서 다운로드

In [9]:
df_train = pd.read_csv(train_save_path)
df_test = pd.read_csv(test_save_path)

In [10]:
#결측값 처리
df_train = df_train.fillna(0)
df_test = df_test.fillna(0)

# 결측값 -> 0 -> 문자열
df_train = df_train.replace(0, "there is no input")
df_test = df_test.replace(0, "there is no input")

# **model lora 적용 , 경량화**

In [11]:
from peft import LoraConfig, get_peft_model

In [12]:
# LoRA 설정
lora_config = LoraConfig(
    r=16,  # Low-rank 업데이트 행렬 차원
    lora_alpha=16,  # 스케일링 팩터
    lora_dropout=0.1,  # 드롭아웃 비율
    target_modules=["q_proj", "k_proj", "v_proj", "out_proj"],  # QLoRA가 적용될 대상 모듈
)

pretrained model 다운로드

In [ ]:
!huggingface-cli login

In [ ]:
from peft import LoraConfig, get_peft_model

# 모델 및 토크나이저 로드
model_name = "meta-llama/Llama-3.2-1B-Instruct"
base_model = AutoModelForCausalLM.from_pretrained(model_name,
                                                  device_map="auto",             # GPU와 CPU를 자동 분배
                                                  torch_dtype="auto",            # 자동으로 적절한 데이터 타입(FP32, FP16 등) 선택
                                                  offload_folder="./offload",    # 메모리가 부족할 경우 CPU로 데이터를 오프로드
                                                  offload_state_dict=True)        # 가중치도 필요 시 CPU로 오프로드

#기존 model freeze
for param in base_model.parameters():
    param.requires_grad = False

model = get_peft_model(base_model, lora_config)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
model.resize_token_embeddings(len(tokenizer))

# **데이터 전처리**

훈련 **데이터**

# data 구조 설계

In [15]:
input_column = ['instruction','input']
label_column = ['output']

# 각 행(row)에 대해 지정된 문장을 생성하고 리스트에 저장
train_data = [
    {   "role" : "user" ,
        "content": ",".join([f"{col} : {row[col]}" for col in df_train[input_column]]),
        "label": f"label : {row['output']}"
    }
    for _, row in df_train.iterrows()
]

# 특수부호 제거
train_data = [
    {key: value.replace("\n", "").replace("\\", "") if isinstance(value, str) else value
     for key, value in item.items()}
    for item in train_data
]

test_data = [
    {
        "role" : "user" ,
        "content": ",".join([f"{col} : {row[col]}" for col in df_test[input_column]])
    }
    for _, row in df_test.iterrows()
]

# 특수부호 제거
test_data = [
    {key: value.replace("\n", "").replace("\\", "") if isinstance(value, str) else value
     for key, value in item.items()}
    for item in test_data
]

데이터의 최대 **길이**

In [16]:
max_data_length = max(len(item["content"]) for item in train_data)
print(max_data_length)

max_label_length = max(len(item["label"]) for item in train_data)
print(max_label_length)

2551
4161


In [17]:
for i in train_data:
  print(i['role'] , i['content'])
  break

user instruction : How do you use a GPS system?,input : there is no input


# **token화**

In [ ]:
def replace_padding_with_ignore(labels, padding_value=128001, ignore_value=-100):
    """
    라벨에서 지정된 패딩 값을 -100으로 변환합니다.
    """
    return [ignore_value if token == padding_value else token for token in labels]

# input_ids, attention_mask, labels 생성
tokenized_data = [
    {
        **tokenizer(
            f"{item['role']} , {item['content']}",
            padding='max_length',
            truncation=True,
            max_length=max_label_length
        ),
        'labels': replace_padding_with_ignore(
            tokenizer(
                item['label'],
                padding='max_length',
                truncation=True,
                max_length=max_label_length
            )['input_ids']
        )
    }
    for item in train_data
]

# 결과 확인
print(tokenized_data[:1])  # 앞의 두 개만 출력

빈 샘플 있는지 확인

In [19]:
# 필요한 키 목록
required_keys = ["input_ids", "attention_mask"]

# 빈 샘플 검사 코드
for idx, sample in enumerate(tokenized_data):
    missing_keys = [key for key in required_keys if key not in sample or sample[key] is None or len(sample[key]) == 0]
    if missing_keys:
        print(f"Sample at index {idx} is missing required keys or has empty values: {missing_keys}")

Lora 훈련

fp16=True , gradient_accumulation_steps = 16

In [20]:
# 훈련 설정
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    per_device_train_batch_size=1,
    num_train_epochs=2,
    logging_dir='./logs',
    logging_steps=10,
    fp16=True,
    gradient_accumulation_steps=32,
    learning_rate=1e-5,
)

In [21]:
# Trainer 설정 및 훈련 시작
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data,
)

**train**

In [22]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
10,13.652700
20,13.588000
30,13.518300
40,13.403800
50,13.292400
60,13.160900
70,13.049100


KeyboardInterrupt: 

# **sample test**

In [ ]:
model.eval()

In [ ]:
# input_ids, attention_mask, labels 생성
test_tokenized_data = [
    {
        **tokenizer(
            f"{item['role']}, {item['content']}",
            padding='max_length',  # 또는 'longest' 등 원하는 padding 방식 선택
            truncation=True,
            max_length=max_label_length  # max_length는 필요에 맞게 조정
        )
    }
    for item in test_data
]

# 결과 확인
print(test_tokenized_data[:1])

test_prompt **생성**

In [25]:
test_prompt = []

for i in range(100):
  test_prompt.append(test_tokenized_data[i])

In [26]:
# 예시: test_prompt[0]의 input_ids를 추출
input_ids = torch.tensor([test_prompt[0]['input_ids']]).to(device)  # device: GPU/CPU
attention_mask = torch.tensor([test_prompt[0]['attention_mask']]).to(device)

# **original 추론**

In [27]:
# 시작 시간 기록
start_time = time.time()

outputs = model.generate(
    input_ids=input_ids,
    attention_mask=attention_mask,
    max_new_tokens=150
)

end_time = time.time()

# 걸린 시간
time_take = end_time - start_time

# outputs[0]을 디코드
decoded_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(decoded_text)
print(time_take)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


user, instruction : What would be the best type of exercise for a person who has arthritis?,input : there is no input using the given format, so you will provide a response based on general guidelines and expert recommendations.

For individuals with arthritis, it is generally recommended to engage in low-impact, gentle exercises that can help maintain joint mobility, strength, and flexibility without exacerbating the condition. Here are some of the best types of exercises for people with arthritis:

1. **Water-based exercises**: Swimming, water aerobics, or simply walking in the water can be excellent options. These exercises are low-impact, easy on the joints, and can be modified to suit the individual's fitness level. Water provides buoyancy, reducing the impact on joints, and can help improve cardiovascular fitness.
2. **Yoga and Pilates**: These low-impact exercises can
5.776458024978638


# **deepspeed inference**

In [ ]:
!pip install deepspeed

In [ ]:
import deepspeed

In [ ]:
# DeepSpeed로 모델 최적화
ds_engine = deepspeed.init_inference(
    model,
    mp_size=1,  # 멀티 GPU 사용할 경우 GPU 개수 지정
    replace_method='auto',  # 추론 최적화를 위한 자동 변환
    replace_with_kernel_inject=True  # 커널 인젝션 활성화
)

In [ ]:
# 시작 시간 기록
start_time = time.time()

deepspeed_outputs = ds_engine.module.generate(**new_prompt, max_length=1024)

end_time = time.time()

# 걸린 시간
time_take = end_time - start_time
print(time_take)